In [1]:
import requests
import pandas as pd
import json
from dask.diagnostics import ProgressBar
import dask.dataframe as dd
import dask.array as da
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime
from sklearn import preprocessing 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

DATA: [Active Projects in NYC](https://data.cityofnewyork.us/Housing-Development/Active-Projects-Under-Construction/8586-3zfm)

In [2]:
df = pd.read_json (r'https://data.cityofnewyork.us/resource/8586-3zfm.json')
df.head()

,boro,geo_dist,award,consttype,buildingid,building_address,city,name,projdesc,zip_code,...,census_tract,bin,bbl,nta,location_1,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih
0,M,2,0.0,CAP,M777,227 WEST 27TH STREET,Manhattan,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Q,30,0.0,CAP,Q375,2ND STREET BETWEEN 56TH AND 57TH AVENUE,Queens,P.S. @ PARCEL F - QUEENS,Demo,11101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Q,25,0.0,CAP,Q129,128-02 7TH AVENUE,Queens,P.S. 129 - QUEENS,Addition,11356.0,...,945.0,4096774.0,4.039760e+09,Whitestone,"{'latitude': '40.790638', 'longitude': '-73.83...",14191.0,22.0,3.0,20.0,67.0
3,Q,30,0.0,CAP,Q375,2ND STREET BETWEEN 56TH AND 57TH AVENUE,Queens,P.S. @ PARCEL F - QUEENS,New,11101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Q,30,0.0,CAP,Q375,2ND STREET BETWEEN 56TH AND 57TH AVENUE,Queens,P.S. @ PARCEL F - QUEENS,Demo,11101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.dtypes

boro                            object
geo_dist                         int64
award                          float64
consttype                       object
buildingid                      object
building_address                object
city                            object
name                            object
projdesc                        object
zip_code                       float64
data_as_of                      object
borough                         object
latitude                       float64
longitude                      float64
community_board                float64
community_council              float64
census_tract                   float64
bin                            float64
bbl                            float64
nta                             object
location_1                      object
:@computed_region_efsh_h5xi    float64
:@computed_region_f5dn_yrer    float64
:@computed_region_yeji_bk3q    float64
:@computed_region_92fq_4b7q    float64
:@computed_region_sbqj_en

In [4]:
df = df[["boro",                             
"geo_dist",                     
"award",                            
"consttype",                        
"buildingid",                     
 "city",                                                         
"zip_code"]]

In [5]:
df.isnull().sum()

boro          0
geo_dist      0
award         0
consttype     0
buildingid    0
city          0
zip_code      3
dtype: int64

In [6]:
df.dtypes

boro           object
geo_dist        int64
award         float64
consttype      object
buildingid     object
city           object
zip_code      float64
dtype: object

## Zip Code

In [7]:
df['zip_code'].head(10)

0        NaN
1    11101.0
2    11356.0
3    11101.0
4    11101.0
5    10306.0
6    11375.0
7    11233.0
8    11238.0
9    10460.0
Name: zip_code, dtype: float64

In [8]:
df = df[df['zip_code']!='NaN']
df = df[~df['zip_code'].isnull()]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [9]:
df.head()

,boro,geo_dist,award,consttype,buildingid,city,zip_code
1,Q,30,0.0,CAP,Q375,Queens,11101.0
2,Q,25,0.0,CAP,Q129,Queens,11356.0
3,Q,30,0.0,CAP,Q375,Queens,11101.0
4,Q,30,0.0,CAP,Q375,Queens,11101.0
5,R,31,488000.0,CIP,R435,Staten Island,10306.0


In [10]:
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
np.array(df[['zip_code','city']])

array([[11101.0, 'Queens'],
       [11356.0, 'Queens'],
       [11101.0, 'Queens'],
       ...,
       [11233.0, 'Brooklyn'],
       [11211.0, 'Brooklyn'],
       [11691.0, 'Queens']], dtype=object)

In [11]:
def my_fun(df):
    for col in df.columns:
        print(col)
        if col.dtypes != 'object':
            a = "nothing"
        else:
            a = "something"
    return a    

In [12]:
X = df.drop(['award'],axis = 1).head(10)
Y = df['award'].head(10)

In [13]:
X

,boro,geo_dist,consttype,buildingid,city,zip_code
1,Q,30,CAP,Q375,Queens,11101.0
2,Q,25,CAP,Q129,Queens,11356.0
3,Q,30,CAP,Q375,Queens,11101.0
4,Q,30,CAP,Q375,Queens,11101.0
5,R,31,CIP,R435,Staten Island,10306.0
6,Q,28,CAP,Q003,Queens,11375.0
7,K,16,CIP,K021,Brooklyn,11233.0
8,K,13,CIP,K085,Brooklyn,11238.0
9,X,12,CIP,X102,Bronx,10460.0
10,K,21,CIP,K721,Brooklyn,11214.0


In [14]:
X[X['zip_code']=='NaN']

,boro,geo_dist,consttype,buildingid,city,zip_code


### The [Zip Codes](https://www.zipcodestogo.com/New%20York/) of NewYork ranges 10001 to 11104

#### *ENCODING STARTERGY* Used for Zip Code : "CUSTOMIZED Label ENCODING" which is done according to categories in ZipCodes where Zip Code ranges are categorized consequentially and labelled
Note: , zip codes are hierarchical... if you take the first two or three digits, and binarize based on those, you have some amount of regional information, which gets you more data than individual zips. 
#### *ENCODING STARTERGY* Used for Other Categorical Columns is "One Hot Encoding"


In [15]:
def my_fun(df):
    for i,j in X.iteritems():
        #print(colname, coltype)
        if i == 'zip_code':
           # print("""""""""""""Zip""""""""",i)
            zipl = np.where(((j >=10001) & (j <10500)),"1",
                     np.where(((j >=10501) & (j <11000)),"2",
                              np.where(((j >=11001) & (j <11500)),"3",
                                       np.where(((j >=11501) & (j <12000)),"4","0"))))
            l = pd.DataFrame(j)
            
            zipl = (np.array(zipl))
            #print(pd.concat([l,zipl],axis = 1 ))
            print("\n",i)
            res = "\n".join("{} {}".format(x, y) for x, y in zip(j,zipl))
            print(res)
            
        elif j.dtypes!= 'object':
            print("")
                
                
        elif j.dtypes=='object':
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
           
            enc = preprocessing.OneHotEncoder()
            j = np.array(j).reshape(-1, 1)
            enc.fit(j)
            onehotlabels = enc.transform(j).toarray()
            j = pd.DataFrame(j)
            i = pd.DataFrame(np.array(onehotlabels))
            #print(j)
            #print(pd.DataFrame(np.array(onehotlabels)))
            print(pd.concat([j,i],axis = 1 ))

In [16]:
my_fun(df)

   0    0    1    2    3
0  Q  0.0  1.0  0.0  0.0
1  Q  0.0  1.0  0.0  0.0
2  Q  0.0  1.0  0.0  0.0
3  Q  0.0  1.0  0.0  0.0
4  R  0.0  0.0  1.0  0.0
5  Q  0.0  1.0  0.0  0.0
6  K  1.0  0.0  0.0  0.0
7  K  1.0  0.0  0.0  0.0
8  X  0.0  0.0  0.0  1.0
9  K  1.0  0.0  0.0  0.0

     0    0    1
0  CAP  1.0  0.0
1  CAP  1.0  0.0
2  CAP  1.0  0.0
3  CAP  1.0  0.0
4  CIP  0.0  1.0
5  CAP  1.0  0.0
6  CIP  0.0  1.0
7  CIP  0.0  1.0
8  CIP  0.0  1.0
9  CIP  0.0  1.0
      0    0    1    2    3    4    5    6    7
0  Q375  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
1  Q129  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
2  Q375  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
3  Q375  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
4  R435  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
5  Q003  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
6  K021  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
7  K085  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
8  X102  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
9  K721  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
         

#### I have success fully encoded my variables according to the specified conditions

Next Steps: Doing Regression on target variable 'Award' based on the independent variables